# Numerical experiments

This notebook produces the data for all tables and figures presented in the numerical experiment section of the paper.

### Preliminaries

In [1]:
%cd -q ..
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']

In [2]:
from contextlib import suppress
from statistics import geometric_mean

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src.classes import FORMULATIONS, ProblemData, Result

In [3]:
np.seterr(divide="ignore", invalid="ignore");

In [4]:
# For final compilation
mpl.rcParams["text.usetex"] = True
plt.style.use("grayscale")

# Instances

In [5]:
experiments = pd.read_csv("instances/instances.csv", index_col="name")
experiments.head()

,group,ratio,num_nodes,num_arcs,num_commodities,num_scenarios
name,,,,,,
r04-7-128,r04,7,10,60,10,128
r04-7-256,r04,7,10,60,10,256
r04-7-512,r04,7,10,60,10,512
r04-7-16,r04,7,10,60,10,16
r04-7-32,r04,7,10,60,10,32


An overview of the base data for each group.

In [6]:
groups = experiments.groupby(experiments.group).first().drop(["ratio"], axis=1)
groups

,num_nodes,num_arcs,num_commodities,num_scenarios
group,,,,
r04,10,60,10,128
r05,10,60,25,128
r06,10,60,50,128
r07,10,82,10,128
r08,10,83,25,128
r09,10,83,50,128
r10,20,120,40,128


In [7]:
groups["num_scenarios"] = ", ".join(
    str(s) for s in sorted(experiments.num_scenarios.unique())
)

print(
    groups.set_axis(["$|N|$", "$|A|$", "$|K|$", "$|S|$"], axis=1)
    .rename(index=str.upper)
    .to_latex()
)

\begin{tabular}{lrrrl}
\toprule
 & $|N|$ & $|A|$ & $|K|$ & $|S|$ \\
group &  &  &  &  \\
\midrule
R04 & 10 & 60 & 10 & 16, 32, 64, 128, 256, 512 \\
R05 & 10 & 60 & 25 & 16, 32, 64, 128, 256, 512 \\
R06 & 10 & 60 & 50 & 16, 32, 64, 128, 256, 512 \\
R07 & 10 & 82 & 10 & 16, 32, 64, 128, 256, 512 \\
R08 & 10 & 83 & 25 & 16, 32, 64, 128, 256, 512 \\
R09 & 10 & 83 & 50 & 16, 32, 64, 128, 256, 512 \\
R10 & 20 & 120 & 40 & 16, 32, 64, 128, 256, 512 \\
\bottomrule
\end{tabular}



# Results

We will first set up some basics to load the data and present some simple reports.

In [8]:
def load(where):
    data = []
    results = []

    for experiment in experiments.index:
        data_loc = f"instances/{experiment}.ndp"
        datum = ProblemData.from_file(data_loc)
        data.append(dict(experiment=experiment, data=datum))

        for formulation in list(FORMULATIONS.keys()) + ["DEQ"]:
            with suppress(FileNotFoundError):
                res_loc = f"{where}/{formulation}/{experiment}.json"
                res = Result.from_file(res_loc)

                results.append(
                    dict(
                        experiment=experiment,
                        formulation=formulation,
                        result=res,
                    )
                )

    results = pd.DataFrame.from_records(results)
    results = results.join(experiments, on="experiment")

    data = pd.DataFrame.from_records(data)
    df = results.merge(data, on="experiment")

    # Additional performance measures that can be derived from the
    # results objects.
    df["run_time"] = df.result.apply(lambda res: res.run_time)
    df["num_iters"] = df.result.apply(lambda res: res.num_iters)
    df["objective"] = df.result.apply(lambda res: res.objective)

    return df

In [22]:
def make_report(df):
    pvt = pd.pivot(
        index="experiment",
        columns="formulation",
        values="run_time",
        data=df,
    )

    num_fastest = pvt.idxmin(axis=1).value_counts()
    num_solved = df.experiment.groupby(df.formulation).count()
    num_formulations = df.formulation.nunique()

    res = pd.merge(
        num_solved,
        num_fastest,
        left_index=True,
        right_index=True,
        how="left",
    )

    res.columns = ["# solved", "# fastest"]

    # Speed-up in run-times over the BB formulation, for instances
    # both BB and the other formulations solve.
    speed_up = pvt.values[:, 0][:, None] / pvt.values
    fn = lambda row: geometric_mean(row[~np.isnan(row)])
    res["speed-up"] = [fn(speed_up[:, idx]) for idx in range(num_formulations)]

    return res.fillna(0)

In [23]:
def make_table(df):
    grouped = (
        df[["num_iters", "run_time"]]
        .groupby([df.group, df.formulation])
        .agg({"num_iters": ["mean", "count"], "run_time": "mean"})
        .set_axis(["No. iters", "Solved", "Time"], axis=1)
    )

    res = (
        grouped.unstack(1)
        .swaplevel(axis=1)
        .sort_index(axis=1)
        .rename(index=str.upper)
    )

    res.columns.names = ["", ""]
    res.index.name = ""

    return res

## Single-commodity instances

This section analyses the results for the single-commodity instances.

In [24]:
df = load("results/single-commodity")

In [25]:
make_report(df).style.format(precision=1)

,# solved,# fastest,speed-up
formulation,,,
BB,126,13,1.0
FlowMIS,126,64,14.1
MIS,126,17,10.1
SNC,126,32,10.9


In [26]:
table = make_table(df)
table.style.format(precision=2)

In [27]:
print(
    table.style.format(precision=1)
    .format_index(escape="latex", axis=1)
    .to_latex(multicol_align="l", hrules=True)
)

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
 & \multicolumn{3}{l}{BB} & \multicolumn{3}{l}{FlowMIS} & \multicolumn{3}{l}{MIS} & \multicolumn{3}{l}{SNC} \\
 & No. iters & Solved & Time & No. iters & Solved & Time & No. iters & Solved & Time & No. iters & Solved & Time \\
 &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
R04 & 25.6 & 18 & 69.8 & 9.2 & 18 & 3.6 & 10.2 & 18 & 6.4 & 10.1 & 18 & 3.7 \\
R05 & 32.7 & 18 & 52.8 & 13.2 & 18 & 3.1 & 14.6 & 18 & 5.5 & 15.0 & 18 & 4.3 \\
R06 & 23.8 & 18 & 61.0 & 10.1 & 18 & 4.1 & 9.7 & 18 & 4.4 & 9.6 & 18 & 2.9 \\
R07 & 56.9 & 18 & 169.6 & 14.1 & 18 & 15.1 & 13.4 & 18 & 10.0 & 14.8 & 18 & 17.3 \\
R08 & 14.5 & 18 & 10.7 & 10.9 & 18 & 1.4 & 12.0 & 18 & 1.7 & 12.5 & 18 & 1.7 \\
R09 & 48.4 & 18 & 232.1 & 12.6 & 18 & 12.2 & 11.8 & 18 & 4.9 & 11.7 & 18 & 4.5 \\
R10 & 61.6 & 18 & 328.0 & 15.4 & 18 & 17.0 & 17.6 & 18 & 18.9 & 18.4 & 18 & 21.6 \\
\bottomrule
\end{tabular}



## No enhancements: no VI's, no metric inequalities

This section analyses the results obtained when no further enhancements are in place.
So just the different formulations, in isolation.

In [28]:
df = load("results/no-metric-no-vis")

In [29]:
make_report(df).style.format(precision=1)

,# solved,# fastest,speed-up
formulation,,,
BB,83,0.0,1.0
DEQ,81,1.0,0.9
FlowMIS,116,30.0,14.4
MIS,118,37.0,25.1
SNC,117,50.0,27.4


In [30]:
table = make_table(df)
table.style.format(precision=2)

In [31]:
print(
    table.style.format(precision=1)
    .format_index(escape="latex", axis=1)
    .to_latex(multicol_align="l", hrules=True)
)

\begin{tabular}{lrrrrrrrrrrrrrrr}
\toprule
 & \multicolumn{3}{l}{BB} & \multicolumn{3}{l}{DEQ} & \multicolumn{3}{l}{FlowMIS} & \multicolumn{3}{l}{MIS} & \multicolumn{3}{l}{SNC} \\
 & No. iters & Solved & Time & No. iters & Solved & Time & No. iters & Solved & Time & No. iters & Solved & Time & No. iters & Solved & Time \\
 &  &  &  &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
R04 & 86.9 & 16 & 1071.4 & 1.0 & 16 & 3409.9 & 33.3 & 18 & 120.5 & 29.1 & 18 & 106.7 & 25.2 & 18 & 76.0 \\
R05 & 62.3 & 18 & 498.7 & 1.0 & 15 & 4358.2 & 17.2 & 18 & 58.3 & 14.2 & 18 & 33.5 & 11.6 & 17 & 22.8 \\
R06 & 106.9 & 12 & 876.9 & 1.0 & 11 & 3653.8 & 83.8 & 15 & 564.6 & 71.0 & 16 & 484.2 & 72.2 & 16 & 556.4 \\
R07 & 165.4 & 7 & 1581.6 & 1.0 & 11 & 6116.6 & 71.8 & 12 & 825.1 & 56.6 & 12 & 727.8 & 65.8 & 12 & 569.6 \\
R08 & 179.2 & 12 & 1396.4 & 1.0 & 9 & 1710.7 & 29.6 & 17 & 187.4 & 21.7 & 18 & 137.7 & 18.4 & 18 & 122.9 \\
R09 & 405.5 & 14 & 2974.1 & 1.0 & 12 & 6750.2 & 21.5 & 18 & 190.8 & 14.8 & 18 & 105

## Only VI's

Here we consider the case where also the valid inequalities are added to the model.

In [32]:
df = load("results/no-metric")

In [33]:
make_report(df).style.format(precision=1)

,# solved,# fastest,speed-up
formulation,,,
BB,114,15,1.0
FlowMIS,119,79,1.7
MIS,119,12,0.9
SNC,120,15,0.9


In [34]:
table = make_table(df)
table.style.format(precision=2)

In [35]:
print(
    table.style.format(precision=1)
    .format_index(escape="latex", axis=1)
    .to_latex(multicol_align="l", hrules=True)
)

\begin{tabular}{lrrrrrrrrrrrr}
\toprule
 & \multicolumn{3}{l}{BB} & \multicolumn{3}{l}{FlowMIS} & \multicolumn{3}{l}{MIS} & \multicolumn{3}{l}{SNC} \\
 & No. iters & Solved & Time & No. iters & Solved & Time & No. iters & Solved & Time & No. iters & Solved & Time \\
 &  &  &  &  &  &  &  &  &  &  &  &  \\
\midrule
R04 & 34.9 & 18 & 130.8 & 18.1 & 18 & 40.7 & 19.3 & 18 & 42.5 & 19.9 & 18 & 61.9 \\
R05 & 3.0 & 18 & 1.0 & 3.0 & 18 & 0.9 & 3.0 & 18 & 1.9 & 3.0 & 18 & 1.7 \\
R06 & 90.4 & 14 & 636.2 & 84.5 & 16 & 689.4 & 59.6 & 16 & 477.1 & 52.5 & 16 & 428.7 \\
R07 & 89.3 & 10 & 518.2 & 65.2 & 13 & 732.9 & 62.2 & 13 & 719.0 & 61.4 & 14 & 1037.1 \\
R08 & 10.1 & 18 & 16.1 & 10.3 & 18 & 21.6 & 10.2 & 18 & 15.8 & 10.3 & 18 & 16.7 \\
R09 & 4.5 & 18 & 3.8 & 4.5 & 18 & 2.6 & 4.5 & 18 & 6.4 & 4.5 & 18 & 6.1 \\
R10 & 15.8 & 18 & 77.0 & 11.7 & 18 & 52.9 & 11.5 & 18 & 114.7 & 11.5 & 18 & 106.2 \\
\bottomrule
\end{tabular}



## Everything

Finally, we consider the case where both valid inequalities and the metric inequalities are added to the model.

In [ ]:
df = load("results/everything")

In [ ]:
make_report(df).style.format(precision=1)

In [ ]:
table = make_table(df)
table.style.format(precision=2)

In [ ]:
print(
    table.style.format(precision=1)
    .format_index(escape="latex", axis=1)
    .to_latex(multicol_align="l", hrules=True)
)